평가문항	상세기준
1. CSR matrix가 정상적으로 만들어졌다.  
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
  
  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
   
   
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.  

# 8-9. 프로젝트 - Movielens 영화 추천 실습
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.  
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.  

+ 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
+ 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
  별점을 시청횟수로 해석해서 생각하겠습니다.
+ 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

## 1) 데이터 준비와 전처리
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.  
  
터미널에서 
```
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip
```

In [1]:
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
import pandas as pd
from pandas import DataFrame

In [2]:
# 다운 받은 데이터 읽기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
print(len(ratings))
ratings.head()

1000209


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


1000209개의 영화 별점 data

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


3 & 4 & 5 점 짜리 영화가 836478개, 84% 정도있고 이것만 분석할 거다.

In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [6]:
# column 이름 중 rating -> count 확인하기
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
print(len(movies))
movies.head() # 여기까지가 전처리

3883


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


영화의 총 개수는 3883개 인데 이게 3, 4, 5점 영화에 포함되어있는지도 확인해야 할 듯

## 2) 분석해 봅시다.
+ ratings에 있는 유니크한 영화 개수
+ rating에 있는 유니크한 사용자 수
+ 가장 인기 있는 영화 30개(인기순)

In [8]:
# ratings movie_id 에 맞쳐서 movies 의 title과 genre 합치기 (merging)
# df_merge_how_outer = pd.merge(df_left, df_right,    ...: how='outer',    ...: on='KEY')
df_merge = pd.merge(ratings, movies, how='left', on='movie_id')
df_merge.head(1000)

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
995,10,2874,5,978227188,"Pajama Game, The (1957)",Comedy
996,10,899,4,979167758,Singin' in the Rain (1952),Musical|Romance
997,10,1135,4,978229435,Private Benjamin (1980),Comedy
998,10,2948,4,978227166,From Russia with Love (1963),Action


In [9]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'count', 'title', 'genre']
df_merge = df_merge[using_cols]
print(len(df_merge))

836478


In [10]:
df_merge.tail(20)

,user_id,movie_id,count,title,genre
836458,6040,527,5,Schindler's List (1993),Drama|War
836459,6040,535,4,Short Cuts (1993),Drama
836460,6040,2010,5,Metropolis (1926),Sci-Fi
836461,6040,2011,4,Back to the Future Part II (1989),Comedy|Sci-Fi
836462,6040,3751,4,Chicken Run (2000),Animation|Children's|Comedy
836463,6040,2019,5,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
836464,6040,541,4,Blade Runner (1982),Film-Noir|Sci-Fi
836465,6040,1077,5,Sleeper (1973),Comedy|Sci-Fi
836466,6040,549,4,Thirty-Two Short Films About Glenn Gould (1993),Documentary
836467,6040,2020,3,Dangerous Liaisons (1988),Drama|Romance


user는 총 6040명

### ratings에 있는 유니크한 영화 개수

In [11]:
# ratings에 있는 유니크한 영화 개수
df_merge['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [12]:
# rating에 있는 유니크한 사용자 수  
df_merge['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [13]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = df_merge.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

Silence of the Lambs, The (1991)  
Jurassic Park (1993)  
Matrix, The (1999)  
Ghostbusters (1984)  
Bug's Life, A (1998)    
좋아하는 영화를 df_merge에 행 추가하기  

In [14]:
movies[movies['title'] == 'Silence of the Lambs, The (1991)']

,movie_id,title,genre
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller


In [15]:
movies[movies['title'] == 'Jurassic Park (1993)']

,movie_id,title,genre
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi


In [16]:
movies[movies['title'] == 'Matrix, The (1999)']

,movie_id,title,genre
2502,2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller


In [17]:
movies[movies['title'] == 'Ghostbusters (1984)']

,movie_id,title,genre
2647,2716,Ghostbusters (1984),Comedy|Horror


In [18]:
movies[movies['title'] == 'Bug\'s Life, A (1998)']

,movie_id,title,genre
2286,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [19]:
df_merge.tail()

,user_id,movie_id,count,title,genre
836473,6040,1090,3,Platoon (1986),Drama|War
836474,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,Sophie's Choice (1982),Drama
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi


In [20]:
df_merge.loc[len(df_merge)] = [6041, 593, 5,'Silence of the Lambs, The (1991)', 'Drama|Thriller']

In [21]:
# 추가 되었는지 확인
df_merge.tail()

,user_id,movie_id,count,title,genre
836474,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,Sophie's Choice (1982),Drama
836477,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
836478,6041,593,5,"Silence of the Lambs, The (1991)",Drama|Thriller


In [22]:
df_merge.loc[len(df_merge)] = [6041, 480, 5, 'Jurassic Park (1993)', 'Action|Adventure|Sci-Fi']
df_merge.loc[len(df_merge)] = [6041, 42571, 5, 'Matrix, The (1999)', 'Action|Sci-Fi|Thriller']
df_merge.loc[len(df_merge)] = [6041, 2716, 5, 'Ghostbusters (1984)', 'Comedy|Horror']
df_merge.loc[len(df_merge)] = [6041, 2355, 5, 'Bug\'s Life, A (1998)', 'Animation|Children\'s|Comedy']

In [23]:
# 한 번 더 확인
df_merge.tail()

,user_id,movie_id,count,title,genre
836478,6041,593,5,"Silence of the Lambs, The (1991)",Drama|Thriller
836479,6041,480,5,Jurassic Park (1993),Action|Adventure|Sci-Fi
836480,6041,42571,5,"Matrix, The (1999)",Action|Sci-Fi|Thriller
836481,6041,2716,5,Ghostbusters (1984),Comedy|Horror
836482,6041,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy


## 4) CSR matrix를 직접 만들어 봅시다.
```
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape
```

In [24]:
num_user = df_merge['user_id'].nunique()
num_movie = df_merge['movie_id'].nunique()
num_user # 내가 추가한 6041

6040

In [25]:
from scipy.sparse import csr_matrix

num_user = df_merge['user_id'].nunique()
num_movie = df_merge['movie_id'].nunique()

csr_data = csr_matrix((df_merge['count'], (df_merge.user_id, df_merge.movie_id))) #?????
csr_data

<6042x42572 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.  
이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다.   
Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.  

In [26]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 학습 내용과는 무관.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 init 파라미터를 살펴보겠습니다.
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지
  
1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [27]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, dtype=np.float32)

In [28]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<42572x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [29]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

##  6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [30]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = df_merge['user_id'].unique()
movie_unique = df_merge['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
print(len(user_unique))
user_unique

6040


array([   1,    2,    3, ..., 6039, 6040, 6041])

### 나와 내가 선호한 Matrix의 내적

In [31]:
user, movie = user_to_idx[6041], movie_to_idx['Matrix, The (1999)']
user_vector, movie_vector = als_model.user_factors[user], als_model.item_factors[movie]

print('ok')

ok


In [32]:
user_vector

array([ 0.4892247 ,  0.1910544 ,  0.3511359 ,  0.25049236,  0.23266591,
       -0.1616118 ,  0.18409292, -0.5182913 ,  0.25813833,  0.6092185 ,
       -0.36334917,  0.28071252,  0.04002229, -0.488594  ,  0.5238275 ,
        1.1529981 ,  0.30129647, -0.71556   , -0.29056314, -0.18550648,
        0.45391494, -0.28430924,  0.64006764, -0.25863034, -0.5709785 ,
       -0.23659714,  0.35324505,  0.36317143,  0.07705693,  0.43531948,
       -0.18820113,  0.43215486, -0.5538832 , -0.46600434, -0.17057998,
        0.22574314,  0.6082949 ,  0.24100658,  0.4477581 , -0.86345047,
        0.16429293, -0.20034753,  0.34718755, -0.27337492,  0.24139445,
        0.1805371 , -0.24316421, -0.23559685, -0.10671976, -0.09986042,
        0.81462485,  0.6926247 , -0.33537337,  0.74698657,  0.26509285,
       -0.44144544, -0.22390592,  0.21528417, -0.4933743 , -0.02960127,
       -0.0433467 ,  0.2624338 , -1.015832  , -0.10702317,  0.00585827,
       -0.7361016 , -0.3743169 ,  0.5103745 ,  0.38254145,  0.51

In [33]:
movie_vector

array([-0.00397174,  0.00123991, -0.00260197,  0.0026197 ,  0.02000462,
        0.0061885 ,  0.00045791,  0.02062204,  0.01425746,  0.01274739,
       -0.00021081,  0.00880046, -0.00678541,  0.01404193,  0.00707805,
        0.00855777, -0.00291977,  0.00147157,  0.00762934,  0.00969866,
        0.01642125,  0.01374933,  0.00995494,  0.00418398,  0.00792021,
        0.0167327 ,  0.00344826,  0.00137268, -0.00394103,  0.00535642,
       -0.00029638, -0.00806616,  0.00274652,  0.00665393,  0.02193982,
       -0.01591994,  0.00020577,  0.00462533,  0.01741555,  0.01312711,
        0.0002115 ,  0.01087   ,  0.00931121,  0.00922474,  0.00396165,
        0.01700873, -0.00421152,  0.02328277,  0.01265012,  0.01273733,
        0.00967403,  0.00148209,  0.00483511,  0.00934074,  0.01262814,
        0.0146325 ,  0.00211728,  0.00049504,  0.0123293 ,  0.01624593,
        0.00603293,  0.00030286, -0.00671536,  0.00131543,  0.00684765,
        0.00863179, -0.00327155,  0.0094522 ,  0.00764862,  0.00

In [34]:
# 내적 
np.dot(user_vector, movie_vector)

0.014019716

### 나와 선택하지 않은 영화와의 내적

In [35]:
movie = movie_to_idx['Sixth Sense, The (1999)']
movie_vector = als_model.item_factors[movie]

print('ok')

ok


In [36]:
# 내적 
np.dot(user_vector, movie_vector)


-0.00891792

선호했던 영화인 Matrix가 훨씬 높게 나왔다. 

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

### Matrix와 비슷한 영화 찾기

In [101]:
movie_id = movie_to_idx['Matrix, The (1999)']
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 1.0000002),
 (614, 0.7332723),
 (2175, 0.71712804),
 (884, 0.70382816),
 (701, 0.6933543),
 (1510, 0.6933338),
 (763, 0.69333166),
 (138, 0.69332945),
 (2821, 0.6933289),
 (3229, 0.6933287),
 (584, 0.6933282),
 (607, 0.693328),
 (2214, 0.69332796),
 (1548, 0.6933278),
 (712, 0.6909411)]

### movie_id로 영화 제목 찾기
#### 조건문으로 df에서  행 추출

In [38]:
movies[movies['movie_id'] == 3380]

,movie_id,title,genre
3311,3380,Railroaded! (1947),Film-Noir


In [39]:
print(movies[movies['movie_id'] == 3380],
     movies[movies['movie_id'] == 614],
     movies[movies['movie_id'] == 701],
     movies[movies['movie_id'] == 2821],
     movies[movies['movie_id'] == 138],
     sep='\n')

      movie_id               title      genre
3311      3380  Railroaded! (1947)  Film-Noir
     movie_id          title           genre
610       614  Loaded (1994)  Drama|Thriller
     movie_id         title  genre
692       701  Daens (1992)  Drama
      movie_id                   title            genre
2752      2821  Male and Female (1919)  Adventure|Drama
     movie_id                   title  genre
136       138  Neon Bible, The (1995)  Drama


#### idx_to_movie 활용하기

In [96]:
idx_to_movie = {v:k for k, v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matrix, The (1999)',
 'Jewel of the Nile, The (1985)',
 'Hellraiser III: Hell on Earth (1992)',
 'Mad Max Beyond Thunderdome (1985)',
 'Santa Claus: The Movie (1985)',
 'Parent Trap, The (1998)',
 'Superman II (1980)',
 'True Lies (1994)',
 'Steam: The Turkish Bath (Hamam) (1997)',
 'Music From Another Room (1998)',
 'Star Trek III: The Search for Spock (1984)',
 'Godfather, The (1972)',
 'Money Train (1995)',
 'Event Horizon (1997)',
 'To Wong Foo, Thanks for Everything! Julie Newmar (1995)']

idx_to_movie를 활용하면 훨씬 짧은 코드로 실행할 수 있다.

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [115]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
recommended_movie = als_model.recommend(user, csr_data, N=15, filter_already_liked_items=True)
recommended_movie

[(910, 1.1678462),
 (3061, 1.0528175),
 (1267, 0.89993477),
 (1035, 0.8963792),
 (3675, 0.8752177),
 (898, 0.8559089),
 (594, 0.8493147),
 (908, 0.8226003),
 (905, 0.80003667),
 (969, 0.786883),
 (2941, 0.78636444),
 (2078, 0.7722172),
 (1196, 0.75479835),
 (1288, 0.7545718),
 (1032, 0.74686897)]

In [116]:
movies[movies['movie_id'] == 910]

,movie_id,title,genre
898,910,Some Like It Hot (1959),Comedy|Crime


In [105]:
print(type(movie_to_idx.items()))

<class 'dict_items'>


In [117]:
# idx_to_movie = {v:k for k, v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in recommended_movie]

KeyError: 3675

In [114]:
idx_to_movie

{0: "One Flew Over the Cuckoo's Nest (1975)",
 1: 'James and the Giant Peach (1996)',
 2: 'My Fair Lady (1964)',
 3: 'Erin Brockovich (2000)',
 4: "Bug's Life, A (1998)",
 5: 'Princess Bride, The (1987)',
 6: 'Ben-Hur (1959)',
 7: 'Christmas Story, A (1983)',
 8: 'Snow White and the Seven Dwarfs (1937)',
 9: 'Wizard of Oz, The (1939)',
 10: 'Beauty and the Beast (1991)',
 11: 'Gigi (1958)',
 12: 'Miracle on 34th Street (1947)',
 13: "Ferris Bueller's Day Off (1986)",
 14: 'Sound of Music, The (1965)',
 15: 'Airplane! (1980)',
 16: 'Tarzan (1999)',
 17: 'Bambi (1942)',
 18: 'Awakenings (1990)',
 19: 'Big (1988)',
 20: 'Pleasantville (1998)',
 21: 'Wallace & Gromit: The Best of Aardman Animation (1996)',
 22: 'Back to the Future (1985)',
 23: "Schindler's List (1993)",
 24: 'Meet Joe Black (1998)',
 25: 'Pocahontas (1995)',
 26: 'E.T. the Extra-Terrestrial (1982)',
 27: 'Titanic (1997)',
 28: 'Ponette (1996)',
 29: 'Close Shave, A (1995)',
 30: 'Antz (1998)',
 31: 'Girl, Interrupted (199

**2) 분석해 봅시다.**  
ratings에 있는 유니크한 영화 개수  
rating에 있는 유니크한 사용자 수  
가장 인기 있는 영화 30개(인기순)  
  
**3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.  
4) CSR matrix를 직접 만들어 봅시다.  
5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.   
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.  
7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.  
8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.**  
   
   
평가문항	상세기준
1. CSR matrix가 정상적으로 만들어졌다.  
사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.  
사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
   
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.  
MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.  
